<a href="https://colab.research.google.com/github/bellomusodiq/machine_learning/blob/master/cats_vs_dogs_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, models, transforms
import numpy as np
import matplotlib.pyplot as plt
import time

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# from pyunpack import Archive
# Archive('/content/drive/My Drive/dataset.rar').extractall('/content/drive/My Drive/cats_vs_dogs')

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
mean = np.array([.485, .456, .406])
std = np.array([.229, .224, .225])

In [0]:
data_transforms = {
    'train': transforms.Compose([
      transforms.RandomResizedCrop(160),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(160),
      transforms.ToTensor(),
      transforms.Normalize(mean, std)
    ])
}

In [0]:
data_dir = '/content/drive/My Drive/cats_vs_dogs/dataset/'
sets = ['train', 'val']

In [0]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

In [0]:
data_loaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                               shuffle=True, num_workers=2) 
                for x in ['train', 'val']}

In [0]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [0]:
class_name = image_datasets['train'].classes
class_name

['cats', 'dogs']

In [0]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 18, 5)
        self.fc1 = nn.Linear(18 * 37 * 37, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        # print(x.shape)
        x = x.view(-1, 18 * 37 * 37)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x

In [0]:
network = ConvNet().to(device)

lr = 0.01
n_epochs = 20

optimizer = torch.optim.Adam(network.parameters(), lr=lr, weight_decay=.01)

In [0]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [0]:

# for epoch in range(n_epochs):

#   total_loss = 0
#   total_correct = 0

#   for batch in data_loaders['train']:
#     images, labels = batch

#     images = images.to(device)
#     labels = labels.to(device)

#     preds = network(images)
#     loss = F.cross_entropy(preds, labels)

#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

#     total_loss += loss.item()
#     total_correct += get_num_correct(preds, labels)

#   print("epoch:", epoch+1, "accuracy:", total_correct/len(data_loaders['train']), 
#         "loss:", total_loss)
#   with torch.no_grad():
#     total_loss = 0
#     total_correct = 0
#     for batch in data_loaders['val']:
#       images, labels = batch
#       images = images.to(device)
#       labels = labels.to(device)
#       preds = network(images)
#       loss = F.cross_entropy(preds, labels)

#       total_loss += loss.item()
#       total_correct += get_num_correct(preds, labels)
#     print("val_accuracy:", total_correct/len(data_loaders['val']), 
#       "val_loss:", total_loss)
#     print("="*200)

In [0]:
model = models.mobilenet_v2(pretrained=True)
model = model.to(device)

In [0]:
for params in model.parameters():
  params.required_grad = False

In [0]:
model.classifier[1] = nn.Linear(1280, 2).to(device)

In [0]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10

for epoch in range(epochs):
  total_loss = 0
  total_correct = 0
  model.train()
  for i, batch in enumerate(data_loaders['train']):
    images, labels = batch

    images = images.to(device)
    labels = labels.to(device)

    preds = model(images)

    loss = F.cross_entropy(preds, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    total_correct += get_num_correct(preds, labels)
    # print('Batch:', i+1)
  print('Epoch:', epoch+1, 'Accuracy:',total_correct/8000, 'loss:', total_loss)
  with torch.no_grad():
    total_correct = 0
    total_loss = 0

    for i, batch in enumerate(data_loaders['val']):
      images, labels = batch
      images = images.to(device)
      labels = labels.to(device)

      preds = model(images)
      # print(preds, labels)
      total_correct += get_num_correct(preds, labels)

      total_loss += F.cross_entropy(preds, labels).item()
    print('val_Accuracy:',total_correct/2000, 'val_loss:', total_loss)


Epoch: 1 Accuracy: 0.863 loss: 76.28721474856138
val_Accuracy: 0.8975 val_loss: 15.675195276737213
Epoch: 2 Accuracy: 0.88875 loss: 62.5226951315999
val_Accuracy: 0.9405 val_loss: 9.746228214353323
Epoch: 3 Accuracy: 0.901375 loss: 57.37734595686197
val_Accuracy: 0.936 val_loss: 9.895741950720549
Epoch: 4 Accuracy: 0.91 loss: 52.66276906430721
Epoch: 5 Accuracy: 0.909625 loss: 52.42779826372862
val_Accuracy: 0.9405 val_loss: 10.253840684890747
Epoch: 6 Accuracy: 0.91175 loss: 50.55378517508507
val_Accuracy: 0.9535 val_loss: 8.048787355422974
Epoch: 7 Accuracy: 0.913125 loss: 49.670569594949484
val_Accuracy: 0.949 val_loss: 8.381430223584175
Epoch: 8 Accuracy: 0.91625 loss: 48.0724261701107
val_Accuracy: 0.9385 val_loss: 8.940534438937902
Epoch: 9 Accuracy: 0.91925 loss: 46.535815730690956
val_Accuracy: 0.9475 val_loss: 8.844832267612219
Epoch: 10 Accuracy: 0.92075 loss: 45.30597477406263
val_Accuracy: 0.949 val_loss: 7.494812456890941


In [0]:
with torch.no_grad():
  total_correct = 0
  total_loss = 0

  for i, batch in enumerate(data_loaders['val']):
    images, labels = batch
    images = images.to(device)
    labels = labels.to(device)

    preds = model(images)
    print(preds.argmax(dim=1))
    print(labels)
    print('='*200)
    total_correct += get_num_correct(preds, labels)

    total_loss += F.cross_entropy(preds, labels).item()
  print('Accuracy:',total_correct/2000, 'loss:', total_loss)

NameError: ignored